# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [2]:
# some imports

import sys
sys.path.insert(0, "../modules")
import pandas as pd
import cufflinks as cf
import plotly.express as px
import re
import module as mod # functions are include in module.py.
cf.go_offline()

In [76]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.head()

,Causa de muerte,Sexo,Edad,Periodo,Total
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2014,395830


In [77]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2014,395830,001-102,Multiple causes,I-XXII.Todas las causas


In [94]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']
categories = mod.cat_var(deaths, var_list)


In [95]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

In [32]:
#To know the evolution of the cancers, we need causes from 09 to 41:

causes_table = causes_table[causes_table["cause_code"].str.contains("-") == False]
causes_table = causes_table.astype({"cause_code": int})
cancer_type = causes_table[causes_table["cause_code"].between(9,41,inclusive=True)]
cancers=cancer_type['cause_name']


In [92]:
#Get table of deaths filtered only by cancers

df_cancer = mod.row_filter(deaths, 'Sexo', ['Hombres','Mujeres']) #Nos qudamos con hombres y mujeres, quitando el total
df_cancer= mod.row_filter(dataset_sexo, 'Edad', ['Todas las edades']) #Nos quedamos con todas las edades.
df_cancer = mod.row_filter(dataset_sexo, 'cause_name', cancers) 


In [43]:
# We only want to get the moost deadly cancer
dataset_sexo_sorted=dataset_sexo.sort_values('Total').groupby(['Periodo','Sexo']).tail(1) #Nos quedamos con la linea con el total mas grande (la última)
dataset_sexo_periodo=dataset_sexo_sorted.sort_values('Periodo', ascending=True) #Ordenamos la tabla por período
dataset_sexo_periodo.to_csv('../data/cancer_deaths_evolution.csv', index=False)

In [88]:
total_deaths_period = mod.row_filter(deaths, 'Sexo', ['Hombres','Mujeres'])
total_deaths_period= mod.row_filter(total_deaths_period, 'Edad', ['Todas las edades'])
total_deaths_period = total_deaths_period[total_deaths_period["cause_code"].str.contains("-") == False]
total_deaths_period=total_deaths_period.groupby(['Periodo','Sexo']).sum()
total_deaths_period.to_csv('../data/total_deaths_period.csv')

Result graphic in Tableau:

Take into account we only have showing the more deadly cancer

https://public.tableau.com/app/profile/ivan.repilado/viz/cancer_deaths_evolution/cancer_deaths_evolution?publish=yes

